# Squashed Entanglement
$$
 \newcommand{\ul}[1]{\underline{#1}}
 \newcommand{\rvalp}[0]{{\ul{\alpha}}}
\newcommand{\alp}[0]{{\alpha}}
 \newcommand{\rvx}[0]{{\ul{x}}}
\newcommand{\rvy}[0]{{\ul{y}}}
$$

The purpose of this notebook is to show how to use entanglish to
calculate the squashed entanglement of a mixed state (either pure or not pure).

Consider a bipartite system
consisting of two parts labelled
by the random variables $\rvx$ and $\rvy$,
and
described by a density matrix $\rho_{\rvx, \rvy}$.
The squashed entanglement of  such a system
is defined as

$$
E_{\rvx, \rvy}(\rho_{\rvx, \rvy}) =
\frac{1}{2}
\min S(\rvx : \rvy|\rvalp)
\;.
$$
The min()---or infimum()
if one wishes to be more mathematically
precise--is
over all density matrices $\rho_{\rvx, \rvy,\rvalp}$
such that ${\rm tr}_\rvalp \; \rho_{\rvx, \rvy,\rvalp}=
\rho_{\rvx, \rvy}$ with $\rho_{\rvx, \rvy}$ held fixed.
If $\rho_{\rvx, \rvy}$ is a pure state, then
$E_{\rvx, \rvy} = S(\rvx) = S(\rvy)$.
Entanglish-Original-Ref discusses other interesting
properties of squashed entanglement

Entanglish-Original-Ref also describes the algo
used by Entanglish to calculate squashed entanglement. The
algorithm is recursive. 
The number of recursive steps 
can be chosen by the user and is called num_ab_steps (ab stands
for Arimoto-Blahut).
Another parameter of the algorithm is num_hidden_states, which is the number of possible 
$\rvalp$ values.

 
**Entanglish-Original-Ref**
* "A New  Algorithm for Calculating
Squashed Entanglement and a Python Implementation Thereof", by R.R.Tucci

First change your working directory to the entanglish directory in your computer, and add its path to the path environment variable.

In [1]:
import os
import sys
print(os.getcwd())
os.chdir('../../')
print(os.getcwd())
sys.path.insert(0,os.getcwd())

C:\Users\rrtuc\Desktop\backed-up\python-projects\entanglish\entanglish\jupyter_notebooks
C:\Users\rrtuc\Desktop\backed-up\python-projects\entanglish


In [2]:
from entanglish.SymNupState import *
from entanglish.SquashedEnt import *

## pure states (symmetrized n-up states)
Next we construct a symmetrized n-up pure state.
Then we compare the Arimoto-Blahut algo entanglement value to the known 
entanglement value, for various possible
bi-partitions of the set of row axes. As expected, they are equal.

recursion_init is a str that specifies how to
initiate the recursion for Kxy_a. There are currently 2 options for recursion_init, 'eigen' and 'eigen+'. 'eigen+' uses Dxy.num_rows^2 degrees of freedom, which is sufficient but 
may not be necessary. 'eigen' uses only Dxy.num_rows degrees of freedom.
With both 'eigen' and 'eigen+', the recursion converges instantly for pure states. 

In [3]:
num_bits = 4
num_up = 1
dm1 = DenMat(1 << num_bits, tuple([2]*num_bits))
st = SymNupState(num_up, num_bits)
st_vec = st.get_st_vec()
dm1.set_arr_from_st_vec(st_vec)

recursion_init = 'eigen+'
num_ab_steps = 5
print('recursion_init=', recursion_init)
print('num_ab_steps=', num_ab_steps)
ecase = SquashedEnt(dm1, num_ab_steps,
    recursion_init=recursion_init, verbose=True)
print('entang_023: algo value, known value\n',
      ecase.get_entang({0, 2, 3}),
      st.get_known_entang(3))
print('entang_02: algo value, known value\n',
      ecase.get_entang({0, 2}),
      st.get_known_entang(2))
print('entang_1: algo value, known value\n',
      ecase.get_entang({1}),
      st.get_known_entang(1))

recursion_init= eigen+
num_ab_steps= 5

initial norm of Dxy - sum_alp Kxy_alp, should be 0
 1.9364916731037083
--ab step= 0 , entang= 1.343657680745164 , err= 9.609221080530707
--ab step= 1 , entang= 0.5622890520968005 , err= 2.88894728360435e-14
--ab step= 2 , entang= 0.5623351446188088 , err= 3.0419099687070626e-14
--ab step= 3 , entang= 0.5623351446188071 , err= 2.9214728099303714e-14
--ab step= 4 , entang= 0.5623351446188036 , err= 3.4337833019422684e-14
entang_023: algo value, known value
 0.5623351446188036 0.5623351446188083

initial norm of Dxy - sum_alp Kxy_alp, should be 0
 1.9364916731037083
--ab step= 0 , entang= 1.3990482477180441 , err= 10.631926515101192
--ab step= 1 , entang= 0.693084373139774 , err= 4.9503425422539945e-14
--ab step= 2 , entang= 0.693147180559945 , err= 4.336419258522215e-14
--ab step= 3 , entang= 0.6931471805599452 , err= 4.40711800909084e-14
--ab step= 4 , entang= 0.693147180559945 , err= 4.4036184523045754e-14
entang_02: algo value, known value
 0.69

## random density matrices
Next we consider 2 random density matrices (actually,
only their eigenvectors are random. Their eigenvalues are specified by the user.)
For each of those 2 density matrices, we calculate the Arimoto-Blahut algo entanglement value, for various possible
bi-partitions of the set of row axes.

In [4]:
np.random.seed(123)
dm = DenMat(8, (2, 2, 2))
evas_of_dm_list = [
    np.array([.07, .03, .25, .15, .3, .1, .06, .04])
    , np.array([.05, .05, .2, .2, .3, .1, .06, .04])
    ]

recursion_init = 'eigen+'
num_ab_steps = 100
print('recursion_init=', recursion_init)
print('num_ab_steps=', num_ab_steps)
for evas_of_dm in evas_of_dm_list:
    evas_of_dm /= np.sum(evas_of_dm)
    print('***************new dm')
    print('evas_of_dm\n', evas_of_dm)
    dm.set_arr_to_rand_den_mat(evas_of_dm)
    ecase = SquashedEnt(dm, num_ab_steps,
        recursion_init=recursion_init, verbose=True)
    print('ent_02_1=', ecase.get_entang({0, 2}))

recursion_init= eigen+
num_ab_steps= 100
***************new dm
evas_of_dm
 [0.07 0.03 0.25 0.15 0.3  0.1  0.06 0.04]

initial norm of Dxy - sum_alp Kxy_alp, should be 0
 4.941631695831571e-16
--ab step= 0 , entang= 0.44627862085938336 , err= 11.783776127318188
--ab step= 1 , entang= 0.10225319005902113 , err= 5.967010661717765
--ab step= 2 , entang= 0.07746633416023459 , err= 1.3410401237553167
--ab step= 3 , entang= 0.06096793172516683 , err= 0.7068780484068223
--ab step= 4 , entang= 0.05356727575418704 , err= 0.5542379815733416
--ab step= 5 , entang= 0.04735622180299261 , err= 0.507309632171571
--ab step= 6 , entang= 0.041788393916642186 , err= 0.48359759606808017
--ab step= 7 , entang= 0.036701584198523755 , err= 0.46497725154766306
--ab step= 8 , entang= 0.03208788874705941 , err= 0.4426562052507335
--ab step= 9 , entang= 0.027962516814249907 , err= 0.41498253126469986
--ab step= 10 , entang= 0.024311421519050774 , err= 0.3856459464751392
--ab step= 11 , entang= 0.02109259175765759

--ab step= 11 , entang= 0.018202032774187697 , err= 0.4537022201360745
--ab step= 12 , entang= 0.015048361167145148 , err= 0.41455823155934823
--ab step= 13 , entang= 0.012527857110295963 , err= 0.3803191482793381
--ab step= 14 , entang= 0.010507179911288078 , err= 0.3513882941202378
--ab step= 15 , entang= 0.008878003271025219 , err= 0.32695700569738206
--ab step= 16 , entang= 0.007558037986326315 , err= 0.30604333266080563
--ab step= 17 , entang= 0.006483751266064293 , err= 0.28767407795996675
--ab step= 18 , entang= 0.005606828793743189 , err= 0.27015106215706214
--ab step= 19 , entang= 0.0048906496126696905 , err= 0.25522974489639016
--ab step= 20 , entang= 0.004300644281423482 , err= 0.24104163540083784
--ab step= 21 , entang= 0.0038110161909144413 , err= 0.2280850736588284
--ab step= 22 , entang= 0.003401818406295413 , err= 0.21608924843317967
--ab step= 23 , entang= 0.0030573916690977724 , err= 0.20495736935095588
--ab step= 24 , entang= 0.0027655571358317227 , err= 0.1946590471